### Lab: Navigating the Lunar Lander with a Dueling Deep Q Network

### University of Virginia
### Reinforcement Learning
#### Last updated: February 27, 2024

---

#### Instructions:

You will work with the `LunarLander-v2` environment from `gymnasium` in this lab.  

An overview of the environment can be found [here](https://gymnasium.farama.org/).  
If you're curious about the source code, see [here](https://github.com/Farama-Foundation/Gymnasium/blob/main/gymnasium/envs/box2d/lunar_lander.py).

Your mission will be to implement a dueling deep Q -network using PyTorch.  
You might run this on Colab.

There are a few specific tasks outlined below for you to solve.

The bigger tasks will be to:

- Show that the algorithm works to train the agent in the environment
- Run episodes and show the results

**Submission**  
As you will likely have several files including this notebook, you can zip all files and submit.

---

![lunar](https://github.com/jackburke12/applied_reinforcement_learning/blob/main/07_deep_q_networks_extensions2/lunar_lander1.png?raw=1)

#### TOTAL POINTS: 12

---

**Hint:** Modules you may need to install include:

swig  
gym[box2d]  
gymnasium

#### 1) What is the penalty for crashing?  
**(POINTS: 1)**


-100 is the penalty for crashing.

#### 2) Set up the environment and run 2 steps by taking random actions.
**(POINTS: 1)**

In [7]:
import gymnasium as gym

env = gym.make("LunarLander-v3", render_mode="human")
obs, info = env.reset()
print("Initial Observation: ", obs)

for step in range(2):
    action = env.action_space.sample()
    next_obs, reward, terminated, truncated, info = env.step(action)
    print(f"Step {step+1}")
    print("Action:", action)
    print("Next obs:", next_obs)
    print("Reward:", reward, "Terminated:", terminated, "Truncated:", truncated)

Initial Observation:  [-0.0053813   1.4043136  -0.5450865  -0.29364684  0.00624241  0.12347026
  0.          0.        ]
Step 1
Action: 1
Next obs: [-0.0108407   1.3971329  -0.55409133 -0.31919786  0.01429689  0.16110553
  0.          0.        ]
Reward: -2.151115201650667 Terminated: False Truncated: False
Step 2
Action: 0
Next obs: [-0.01630039  1.3893526  -0.55411613 -0.3458882   0.02234662  0.16100982
  0.          0.        ]
Reward: -1.4076542823901264 Terminated: False Truncated: False


#### 3) Briefly discuss your approach to solving the problem  
**(POINTS: 2)**



The environment provides an 8-dimensional continuous state vector and 4 discrete actions: do nothing, fire left, fire main, and fire right. We'll create a dueling deep Q-network to land the module safely.

Our network will consist of an 8-feature input layer that takes the state vector as input. Two connected layers will be split into a value stream, V(s), and advantage stream, A(s,a). This is the core of the dueling network architecture. The streams will then combine to form Q(s,a), the state-action value function.

We will also use a replay buffer to store transitions, and sample from mini-batches for training. A target network will provide stable Q-targets and will be updated periodically from the online policy network.

The train step will compute the temporal-difference loss and update network parameters via gradient descent.

#### 4) Create supporting code files (`.py` format) to create the agent, train, and run episodes
**(POINTS: 6)**

Your code should include:

- **(POINTS: 1)** A class for the dueling DQN agent
- **(POINTS: 1)** An architecture with separate Value and Advantage streams
- **(POINTS: 1)** A method called `forward()` for the forward pass of the algorithm
- **(POINTS: 1)** A replay buffer
- **(POINTS: 1)** A training function
- **(POINTS: 1)** A function to run episodes

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random
from collections import deque

In [20]:
class DuelingDQN(nn.Module):
  def __init__(self, state_dim, action_dim):
    super(DuelingDQN, self).__init__()
    self.shared = nn.Sequential(
        nn.Linear(state_dim, 128),
        nn.ReLU(),
        nn.Linear(128,128),
        nn.ReLU()
    )

    self.value_stream = nn.Sequential(
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,1)
    )

    self.advantage_stream = nn.Sequential(
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64, action_dim)
    )

  def forward(self, state):
    x = self.shared(state)
    value = self.value_stream(x)
    advantage = self.advantage_stream(x)
    #combine value and advantage streams. An average of the advantage function is used, like the dueling q-network paper
    q_values = value + (advantage - advantage.mean(dim=1, keepdim = True))
    return q_values

class ReplayBuffer:
  def __init__(self, capacity):
    self.buffer = deque(maxlen=capacity)

  def push(self, state, action, reward, next_state, done):
    self.buffer.append((state, action, reward, next_state, done))

  def sample(self, batch_size):
    batch = random.sample(self.buffer, batch_size)
    states, actions, rewards, next_states, dones = map(np.array, zip(*batch))
    return (
        torch.FloatTensor(states),
        torch.LongTensor(actions),
        torch.FloatTensor(rewards),
        torch.FloatTensor(next_states),
        torch.FloatTensor(dones)
    )

  def __len__(self):
    return len(self.buffer)

def train_step(policy_net, target_net, optimizer, replay_buffer, batch_size, gamma):
  if len(replay_buffer) < batch_size:
    return None

  states, actions, rewards, next_states, dones = replay_buffer.sample(batch_size)

  q_values = policy_net(states).gather(1, actions.unsqueeze(1)).squeeze(1)

  with torch.no_grad():
    max_next_q = target_net(next_states).max(1)[0]
    target = rewards + gamma * max_next_q * (1-dones)

  loss = F.mse_loss(q_values, target)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  return loss.item()

def run_episodes(
    env_name = "LunarLander-v3",
    num_episodes = 100,
    gamma = 0.99,
    lr = 1e-3,
    batch_size = 64,
    buffer_capacity = 100000,
    epsilon_start = 1.0,
    epsilon_end = 0.01,
    epsilon_decay = 0.995,
    target_update_freq = 10
):

  env = gym.make(env_name)
  state_dim = env.observation_space.shape[0]
  action_dim = env.action_space.n

  policy_net = DuelingDQN(state_dim, action_dim)
  target_net = DuelingDQN(state_dim, action_dim)
  target_net.load_state_dict(policy_net.state_dict())
  target_net.eval()

  optimizer = torch.optim.Adam(policy_net.parameters(), lr=lr)
  replay_buffer = ReplayBuffer(buffer_capacity)

  epsilon = epsilon_start
  rewards_history = []

  for episode in range(num_episodes):
    state, info = env.reset()
    total_reward = 0
    done = False

    while not done:
      if np.random.rand() < epsilon:
        action = env.action_space.sample()
      else:
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        q_values = policy_net(state_tensor)
        action = q_values.argmax().item()

      next_state, reward, terminated, truncated, info = env.step(action)
      done = terminated or truncated

      replay_buffer.push(state, action, reward, next_state, done)
      state = next_state
      total_reward += reward

      loss = train_step(policy_net, target_net, optimizer, replay_buffer, batch_size, gamma)

    if episode % target_update_freq == 0:
      target_net.load_state_dict(policy_net.state_dict())

    epsilon = max(epsilon_end, epsilon * epsilon_decay)
    rewards_history.append(total_reward)
    print(f"Episode {episode+1}/{num_episodes} | Reward: {total_reward:.1f} | Epsilon: {epsilon:.3f}")

  env.close()
  return rewards_history

#### 5) Run the training and show evidence that the agent is learning.  

For example, its average reward (score) should increase with more episodes.

**(POINTS: 1 if successful)**

In [23]:
rewards = run_episodes(num_episodes=100)

Episode 1/100 | Reward: -158.4 | Epsilon: 0.995
Episode 2/100 | Reward: -159.6 | Epsilon: 0.990
Episode 3/100 | Reward: -129.1 | Epsilon: 0.985
Episode 4/100 | Reward: -128.4 | Epsilon: 0.980
Episode 5/100 | Reward: -260.8 | Epsilon: 0.975
Episode 6/100 | Reward: -229.2 | Epsilon: 0.970
Episode 7/100 | Reward: -97.0 | Epsilon: 0.966
Episode 8/100 | Reward: -249.1 | Epsilon: 0.961
Episode 9/100 | Reward: -63.4 | Epsilon: 0.956
Episode 10/100 | Reward: -121.2 | Epsilon: 0.951
Episode 11/100 | Reward: -192.5 | Epsilon: 0.946
Episode 12/100 | Reward: -87.5 | Epsilon: 0.942
Episode 13/100 | Reward: -105.4 | Epsilon: 0.937
Episode 14/100 | Reward: -47.7 | Epsilon: 0.932
Episode 15/100 | Reward: -132.0 | Epsilon: 0.928
Episode 16/100 | Reward: -48.7 | Epsilon: 0.923
Episode 17/100 | Reward: -36.1 | Epsilon: 0.918
Episode 18/100 | Reward: -243.3 | Epsilon: 0.914
Episode 19/100 | Reward: -199.4 | Epsilon: 0.909
Episode 20/100 | Reward: -129.1 | Epsilon: 0.905
Episode 21/100 | Reward: -139.8 | E

#### 6) Run a few episodes and show results
**(POINTS: 1 if successful)**